<a href="https://colab.research.google.com/github/etuckerman/SOCOTEC/blob/main/SOCOTEC_GPU_elliot_tuckerman_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `SOCOTEC` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `SOCOTEC`


In [2]:
# Check for GPU availability
!nvidia-smi

Tue Dec 10 00:09:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# Install required libraries
!pip install -q transformers torch accelerate datasets openai
!pip install -q bitsandbytes

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.6 MB/s eta 0:00:00


In [4]:
# Implement function library as specified
def add(a, b): return a + b
def square(a): return a ** 2
def cube(a): return a ** 3
def greet(name): return f"Hello, {name}!"

# Function mapping dictionary
FUNCTION_MAPPING = {
    'add': add,
    'square': square,
    'cube': cube,
    'greet': greet
}

In [5]:
# Create synthetic training dataset
training_data = [
    {"input": "What is the sum of 1 and 2?", "output": "add(1, 2)"}
]

# Additional diverse prompts
test_cases = training_data + [
    # Basic addition scenarios
    {"input": "What is 7 plus 3?", "output": "add(7, 3)"},
    {"input": "Add 10 to 15", "output": "add(10, 15)"},
    {"input": "How much is 20 added to 22?", "output": "add(20, 22)"},
    {"input": "Add 50 and 30 together", "output": "add(50, 30)"},
    {"input": "What is 18 added to 27?", "output": "add(18, 27)"},

    # Square function scenarios
    {"input": "Calculate the square of 5", "output": "square(5)"},
    {"input": "What is the square of 6?", "output": "square(6)"},
    {"input": "Square 7", "output": "square(7)"},
    {"input": "What is the square of 4?", "output": "square(4)"},
    {"input": "What is the square of 8?", "output": "square(8)"},

    # Cube function scenarios
    {"input": "What's the cube of 2?", "output": "cube(2)"},
    {"input": "Cube the number 4", "output": "cube(4)"},
    {"input": "What is the cube of 3?", "output": "cube(3)"},
    {"input": "What is the cube of 5?", "output": "cube(5)"},
    {"input": "What is the cube of 6?", "output": "cube(6)"},

    # Greeting scenarios
    {"input": "Greet John", "output": "greet('John')"},
    {"input": "Say hello to Sarah", "output": "greet('Sarah')"},
    {"input": "Say hello to Emily", "output": "greet('Emily')"},
    {"input": "Greet Michael", "output": "greet('Michael')"},

    # Multi-function mathematical logic scenarios
    {"input": "What is 2 plus the square of 3?", "output": "add(2, square(3))"},
    {"input": "What is 5 plus the cube of 4?", "output": "add(5, cube(4))"},
    {"input": "Add the square of 3 to 10", "output": "add(square(3), 10)"}
]


In [6]:
def generate_function_call(model, tokenizer, input_text, max_length=150):
    prompt = format_prompt(input_text)

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
      **inputs,
      max_new_tokens=50,  # Dynamically allow additional tokens for response generation
      num_return_sequences=1,
      do_sample=True,
      temperature=0.1
  )


    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Improved regular expression for function call extraction
    function_call_match = re.search(r'Correct Function Call:\s*(\w+\(.*?\))', generated_text)

    if function_call_match:
        function_call = function_call_match.group(1).strip()

        # Preserve original quotes
        # ... (consider keeping quotes as generated)

        # More robust argument parsing using ast.literal_eval
        try:
            args = ast.literal_eval(function_call.split("(")[1].rstrip(")"))
        except (SyntaxError, ValueError):
            args = []

        if not re.match(r'^(add|square|cube|greet)\(', function_call):
            return None

        return function_call
    return None


import re
import ast


def parse_function_call(function_call):
    """
    Safely parse nested function calls using AST to avoid mismatched logic or unsafe eval.
    This will ensure expressions like add(square(3), 10) are safely evaluated.
    """
    try:
        # Use AST literal eval to evaluate function calls safely
        parsed_call = ast.literal_eval(function_call)
        return parsed_call
    except (SyntaxError, ValueError):
        return None


def execute_function_call(function_call):
    """
    Safely execute function calls based on the pre-defined FUNCTION_MAPPING.
    Handles nested calls like add(square(3), 10).
    """
    try:
        # Match function name and arguments
        match = re.match(r"(\w+)\((.*)\)", function_call)
        if not match:
            return "Invalid function call format"

        func_name, args_str = match.groups()
        args = [int(arg.strip()) if arg.isdigit() else arg.strip() for arg in args_str.split(",")]

        if func_name in FUNCTION_MAPPING:
            return FUNCTION_MAPPING[func_name](*args)
        else:
            return f"Unsupported function: {func_name}"

    except Exception as e:
        return f"Error executing function: {str(e)}"


def evaluate_model(model, tokenizer, test_cases):
    results = []
    for case in test_cases:
        function_call = generate_function_call(model, tokenizer, case['input'])
        expected_call = case['output']

        # Execute function calls
        expected_result = execute_function_call(expected_call)
        generated_result = execute_function_call(function_call)

        # Compare execution results
        call_match = expected_result == generated_result

        result = {
            'input': case['input'],
            'generated_call': function_call,
            'expected_call': expected_call,
            'call_match': call_match,
            'expected_result': expected_result,
            'generated_result': generated_result
        }

        # If the match was successful, return execution results directly
        if call_match:
            result['execution_result'] = expected_result
        else:
            result['execution_result'] = {
                'expected': expected_result,
                'generated': generated_result
            }

        results.append(result)

    return results


In [7]:
def format_prompt(input_text):
    """
    Prepares the prompt with only the relevant, concise instructions while ensuring syntax correctness.
    """
    return f"""You convert natural language into Python function calls using ONLY these functions:
- add(a, b): Sum of a and b
- square(a): a squared
- cube(a): a cubed
- greet(name): Returns a greeting for the name

RULES:
1. Use only these functions and their exact signatures.
2. Ensure **correct parenthesis usage** and proper nesting.
3. Handle multi-function logic with strict syntax. Example:
   Input: "Add the square of 3 to 10"
   Correct Call: add(square(3), 10)
4. If the input cannot be logically converted, return:
   'Unable to translate into a valid function call.'

Input: {input_text}
Correct Function Call:"""


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch  # Import PyTorch to verify GPU availability

# Model name with instruction fine-tuning
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

# Check if CUDA is available (GPU availability check)
if torch.cuda.is_available():
    device = 'cuda'
    print("Using GPU for computation.")
else:
    device = 'cpu'
    print("GPU not available, falling back to CPU.")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)  # Move model to GPU if available

# Input
input_text = "What is the sum of 45 and 27?"

# Prepare the prompt
prompt = format_prompt(input_text)  # Ensure your `format_prompt` function generates the correct prompt

# Tokenize the input and move tokens to GPU
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate output using the model
outputs = model.generate(**inputs, max_new_tokens=20)

# Decode the response (move the tensor back from GPU to CPU)
response = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)

# Output the result
print(response)


Using GPU for computation.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 181.06 MiB is free. Process 29062 has 14.57 GiB memory in use. Of the allocated memory 14.47 GiB is allocated by PyTorch, and 1.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import ast
# Run evaluation
evaluation_results = evaluate_model(model, tokenizer, test_cases)


In [ ]:
# Print results
for result in evaluation_results:
    print(f"Input: {result['input']}")
    print(f"Generated Call: {result['generated_call']}")
    print(f"Expected Call: {result['expected_call']}")
    print(f"Call Match: {result['call_match']}")
    if result.get('execution_result'):
        print(f"Execution Result: {result['execution_result']}")
    print("---")

In [ ]:
# Define only the three specific test cases you want to evaluate
custom_test_cases = [
    {"input": "Add the square of 3 to 10", "expected_call": "add(square(3), 10)"},
    {"input": "What is 2 plus the square of 3?", "expected_call": "add(2, square(3))"},
    {"input": "What is 5 plus the cube of 4?", "expected_call": "add(5, cube(4))"}
]

# Run only the three test cases
for test_case in custom_test_cases:
    # Generate the function call
    generated_call = generate_function_call(model, tokenizer, test_case["input"])
    # Execute the generated call safely
    execution_result = execute_function_call(generated_call)

    print(f"Input: {test_case['input']}")
    print(f"Generated Call: {generated_call}")
    print(f"Expected Call: {test_case['expected_call']}")
    print(f"Execution Result: {execution_result}")
    print("---")
